In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# data load
df = pd.read_pickle('/Users/sangjun/Desktop/MSDS_Course/MSDS_601/Final_Project/AfterWrangling.pkl')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [4]:
df.columns = df.columns.str.replace(' ', '_')
df = df.drop(['Churn_Reason', 'CustomerID', 'Country', 'State', 
              'City', 'Lat_Long', 'Count', 'Latitude', 'Longitude', 
              'Churn_Value', 'Zip_Code'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             7043 non-null   object 
 1   Senior_Citizen     7043 non-null   object 
 2   Partner            7043 non-null   object 
 3   Dependents         7043 non-null   object 
 4   Tenure_Months      7043 non-null   int64  
 5   Phone_Service      7043 non-null   object 
 6   Multiple_Lines     7043 non-null   object 
 7   Internet_Service   7043 non-null   object 
 8   Online_Security    7043 non-null   object 
 9   Online_Backup      7043 non-null   object 
 10  Device_Protection  7043 non-null   object 
 11  Tech_Support       7043 non-null   object 
 12  Streaming_TV       7043 non-null   object 
 13  Streaming_Movies   7043 non-null   object 
 14  Contract           7043 non-null   object 
 15  Paperless_Billing  7043 non-null   object 
 16  Payment_Method     7043 

In [5]:
df_cat_index = df.select_dtypes(include=['object', 'category']).columns
df_cat_index

df_numeric_index = df.select_dtypes(include=['int', 'float']).columns
df_numeric_index

Index(['Tenure_Months', 'Monthly_Charges', 'Total_Charges', 'Churn_Score',
       'CLTV'],
      dtype='object')

In [6]:
df_cat = df[df_cat_index]
df_numeric = df[df_numeric_index]

## Check Multicollinearity

In [7]:
# VIF
formula = 'Churn_Label ~ Tenure_Months+Monthly_Charges+Total_Charges+Churn_Score+CLTV'
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
y, X = dmatrices(formula, data=df, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)

   VIF Factor         features
0   34.186041        Intercept
1    5.952947    Tenure_Months
2    3.340829  Monthly_Charges
3    9.611085    Total_Charges
4    1.106657      Churn_Score
5    1.189480             CLTV


Total_Charges is close to 10. It's not a serious multicolinearity problem, but we still have to check later. 

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import train_test_split

def calculate_gvif(df, response_var):
    # Drop rows with any null values
    df = df.dropna()

    # Handle categorical variables by one-hot encoding
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if response_var in categorical_cols:
        categorical_cols = categorical_cols.drop(response_var)
        
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(drop='first', sparse=False)
        encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))
        
        # Manually creating feature names for encoded columns
        feature_names = [f"{col}_{category}" for col, categories in zip(categorical_cols, encoder.categories_) for category in categories[1:]]
        encoded_cols.columns = feature_names

        df = pd.concat([df.drop(categorical_cols, axis=1), encoded_cols], axis=1)

    # Ensure all the predictor variables are numeric
    if response_var in df.columns:
        predictors = df.columns.drop(response_var)
    else:
        print(f"Warning: {response_var} not found in DataFrame. Make sure the response variable is correctly named.")
        return pd.DataFrame(columns=["Feature", "GVIF"])
        
    gvif_data = []

    for feature in predictors:
        try:
            X = df.drop(columns=[feature, response_var])
            y = df[feature]

            # Split data to avoid multicollinearity
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = OLS(y_train, X_train).fit()
            rsquared = model.rsquared
            gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1))) 
            gvif_data.append((feature, gvif))
        except Exception as e:
            print(f"An error occurred when processing the feature {feature}: {e}")
            gvif_data.append((feature, None))

    gvif_df = pd.DataFrame(gvif_data, columns=["Feature", "GVIF"])
    return gvif_df


response_var = 'Churn_Label'  # replace with the name of your response variable
gvif_values = calculate_gvif(df_cat, response_var)

#gvif_values = calculate_gvif(df)
print(gvif_values)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarnin

                                   Feature      GVIF
0                              Gender_Male  1.000033
1                       Senior_Citizen_Yes  1.003147
2                              Partner_Yes  1.006596
3                           Dependents_Yes  1.005621
4                        Phone_Service_Yes  1.065889
5          Multiple_Lines_No phone service  1.018535
6                       Multiple_Lines_Yes  1.007627
7             Internet_Service_Fiber optic  1.017668
8                      Internet_Service_No       inf
9      Online_Security_No internet service       inf
10                     Online_Security_Yes  1.008107
11       Online_Backup_No internet service       inf
12                       Online_Backup_Yes  1.007024
13   Device_Protection_No internet service       inf
14                   Device_Protection_Yes  1.009556
15        Tech_Support_No internet service       inf
16                        Tech_Support_Yes  1.009884
17        Streaming_TV_No internet service    

/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))
/var/folders/yx/vpn4pp6d44z8vryqc0zfxyp40000gn/T/ipykernel_85708/3501188714.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  gvif = (1 / (1 - rsquared))**(1 / (2 * (model.df_model + 1)))


Categorical variables: Internet_Service, Online_Security, Online_Backup, Device_Protection, Tech_Support, Streaming_TV, and Streaming_Movies have serious multicolinearity issue. Therefore, we are taking out these variables.

### After VIF and GVIF

In [9]:
df = df.drop(['Internet_Service', 'Online_Security', 'Online_Backup', 'Device_Protection', 
              'Tech_Support', 'Streaming_TV', 'Streaming_Movies'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             7043 non-null   object 
 1   Senior_Citizen     7043 non-null   object 
 2   Partner            7043 non-null   object 
 3   Dependents         7043 non-null   object 
 4   Tenure_Months      7043 non-null   int64  
 5   Phone_Service      7043 non-null   object 
 6   Multiple_Lines     7043 non-null   object 
 7   Contract           7043 non-null   object 
 8   Paperless_Billing  7043 non-null   object 
 9   Payment_Method     7043 non-null   object 
 10  Monthly_Charges    7043 non-null   float64
 11  Total_Charges      7043 non-null   float64
 12  Churn_Label        7043 non-null   object 
 13  Churn_Score        7043 non-null   int64  
 14  CLTV               7043 non-null   int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 825.5+ KB


## Check Outliers

In [10]:
df['Churn_Label']

0       Yes
1       Yes
2       Yes
3       Yes
4       Yes
       ... 
7038     No
7039     No
7040     No
7041     No
7042     No
Name: Churn_Label, Length: 7043, dtype: object

In [11]:
df['Churn_Label'] = df['Churn_Label'].astype('category').cat.codes

In [12]:
print(df.dtypes)

Gender                object
Senior_Citizen        object
Partner               object
Dependents            object
Tenure_Months          int64
Phone_Service         object
Multiple_Lines        object
Contract              object
Paperless_Billing     object
Payment_Method        object
Monthly_Charges      float64
Total_Charges        float64
Churn_Label             int8
Churn_Score            int64
CLTV                   int64
dtype: object


In [13]:
df = pd.get_dummies(df, columns=['Gender', 'Senior_Citizen', 'Partner',
                                 'Dependents','Phone_Service','Multiple_Lines',
                                'Contract', 'Paperless_Billing', 'Payment_Method'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tenure_Months                             7043 non-null   int64  
 1   Monthly_Charges                           7043 non-null   float64
 2   Total_Charges                             7043 non-null   float64
 3   Churn_Label                               7043 non-null   int8   
 4   Churn_Score                               7043 non-null   int64  
 5   CLTV                                      7043 non-null   int64  
 6   Gender_Female                             7043 non-null   uint8  
 7   Gender_Male                               7043 non-null   uint8  
 8   Senior_Citizen_No                         7043 non-null   uint8  
 9   Senior_Citizen_Yes                        7043 non-null   uint8  
 10  Partner_No                          

In [14]:
# Pearson Residual

X = df.drop(columns=['Churn_Label'])
y = df['Churn_Label']


X = sm.add_constant(X)  # Adds a constant term to the predictor

# Fit model
model = sm.Logit(y, X)
result = model.fit()

# Calculate Pearson residuals
pearson_residuals = result.resid_pearson

# Set a threshold to identify an outlier
threshold = 3   # This is an arbitrary threshold, adjust according to your analysis

# Get the indices of the outliers
outlier_indices = np.where(np.abs(pearson_residuals) > threshold)

# Print the indices of the outliers
print("Indices of outliers:", outlier_indices)

         Current function value: 0.175861
         Iterations: 35
Indices of outliers: (array([  22,   65,  142,  157,  165,  369,  423,  504,  676,  757,  879,
        883,  934,  994, 1084, 1088, 1121, 1147, 1243, 1254, 1261, 1378,
       1414, 1445, 1457, 1532, 1572, 1584, 1589, 1658, 1696, 1700, 1701,
       1721, 1800, 1855, 2086, 2267, 2896, 3559, 3589, 3615, 3922, 3981,
       3999, 4012, 4018, 4344, 4355, 4518, 4624, 4714, 4870, 5284, 5570,
       5580, 5676, 6145, 6169, 6228, 6811, 6998]),)


/opt/homebrew/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [15]:
# Deviance Residual

# Prepare data
X = df.drop(columns=['Churn_Label'])
y = df['Churn_Label']
X = sm.add_constant(X)  # Adds a constant term to the predictor

# Fit model
model = sm.Logit(y, X)
result = model.fit()

# Calculate Deviance residuals
deviance_residuals = result.resid_dev

# Set a threshold to identify an outlier
threshold = 3   # This is an arbitrary threshold, adjust according to your analysis

# Get the indices of the outliers
outlier_indices = np.where(np.abs(deviance_residuals) > threshold)

# Print the indices of the outliers
print("Indices of outliers based on deviance residuals:", outlier_indices)

         Current function value: 0.175861
         Iterations: 35
Indices of outliers based on deviance residuals: (array([1261]),)


/opt/homebrew/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Based on the two results, we decided to take out index 1261.

In [16]:
df = df.drop(1261, errors='ignore')  
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tenure_Months                             7042 non-null   int64  
 1   Monthly_Charges                           7042 non-null   float64
 2   Total_Charges                             7042 non-null   float64
 3   Churn_Label                               7042 non-null   int8   
 4   Churn_Score                               7042 non-null   int64  
 5   CLTV                                      7042 non-null   int64  
 6   Gender_Female                             7042 non-null   uint8  
 7   Gender_Male                               7042 non-null   uint8  
 8   Senior_Citizen_No                         7042 non-null   uint8  
 9   Senior_Citizen_Yes                        7042 non-null   uint8  
 10  Partner_No                          

In [17]:
import pandas as pd
from scipy.stats import pointbiserialr


# Calculating and printing the point-biserial correlation coefficients for each predictor
for column in df.columns:
    if column != 'y':  # Exclude the outcome variable
        corr, _ = pointbiserialr(df['Churn_Label'], df[column])
        print(f'Correlation between y and {column}: {corr:.2f}')


Correlation between y and Tenure_Months: -0.35
Correlation between y and Monthly_Charges: 0.19
Correlation between y and Total_Charges: -0.20
Correlation between y and Churn_Label: 1.00
Correlation between y and Churn_Score: 0.66
Correlation between y and CLTV: -0.13
Correlation between y and Gender_Female: 0.01
Correlation between y and Gender_Male: -0.01
Correlation between y and Senior_Citizen_No: -0.15
Correlation between y and Senior_Citizen_Yes: 0.15
Correlation between y and Partner_No: 0.15
Correlation between y and Partner_Yes: -0.15
Correlation between y and Dependents_No: 0.25
Correlation between y and Dependents_Yes: -0.25
Correlation between y and Phone_Service_No: -0.01
Correlation between y and Phone_Service_Yes: 0.01
Correlation between y and Multiple_Lines_No: -0.03
Correlation between y and Multiple_Lines_No phone service: -0.01
Correlation between y and Multiple_Lines_Yes: 0.04
Correlation between y and Contract_Month-to-month: 0.41
Correlation between y and Contract

Based on the correlation, we have to delete 'Gender', 'Phone_Service', and 'Multiple_Lines'.

In [19]:
df = df.drop(['Gender_Male', 'Gender_Female', 'Phone_Service_No', 'Phone_Service_Yes'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 24 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tenure_Months                             7042 non-null   int64  
 1   Monthly_Charges                           7042 non-null   float64
 2   Total_Charges                             7042 non-null   float64
 3   Churn_Label                               7042 non-null   int8   
 4   Churn_Score                               7042 non-null   int64  
 5   CLTV                                      7042 non-null   int64  
 6   Senior_Citizen_No                         7042 non-null   uint8  
 7   Senior_Citizen_Yes                        7042 non-null   uint8  
 8   Partner_No                                7042 non-null   uint8  
 9   Partner_Yes                               7042 non-null   uint8  
 10  Dependents_No                       

In [20]:
df['Churn_Label'].value_counts()

0    5174
1    1868
Name: Churn_Label, dtype: int64

In [21]:
X = df.loc[:, df.columns != 'Churn_Label']
y = df.loc[:, df.columns == 'Churn_Label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [22]:
#start = time.process_time()
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(solver='newton-cg')
model_res = model.fit(X_train, y_train.values.ravel())
y_pred = model_res.predict(X_test)

# evaluate model
scores = cross_val_score(model, y_test, y_pred, scoring='f1', cv=cv, n_jobs=-1)

# print time consumption
#lr_time = time.process_time() - start

# report performance
print('F1-Score: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

/opt/homebrew/anaconda3/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/homebrew/anaconda3/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


F1-Score: 0.853 (0.036)


In [26]:
from sklearn.metrics import log_loss
import math


# Number of parameters (features + intercept)
k = X.shape[1] + 1

# Log-likelihood of the model
y_pred = model.predict_proba(X)
log_likelihood = -log_loss(y, y_pred, normalize=False)

# Calculate AIC and BIC
AIC = 2*k - 2*log_likelihood
BIC = -2*log_likelihood + k*math.log(X.shape[0])

print(f"AIC: {AIC}")
print(f"BIC: {BIC}")

AIC: 2515.181089819441
BIC: 2679.812629812601


In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import math
from itertools import combinations

# Sample data
data = pd.DataFrame({
    'x1': np.random.rand(100),
    'x2': np.random.rand(100),
    'x3': np.random.rand(100),
    'y': np.random.randint(0, 2, 100)  # Generating a random binary target variable
})

# Function to calculate AIC and BIC for multiple combinations of features
def calculate_aic_bic(data, target, feature_combinations):
    results = []
    
    for features in feature_combinations:
        X = data[list(features)]
        y = data[target]

        # Fit logistic regression model
        model = LogisticRegression()
        model.fit(X, y)

        # Number of parameters (features + intercept)
        k = len(features) + 1

        # Calculate log-likelihood
        y_pred = model.predict_proba(X)
        log_likelihood = -log_loss(y, y_pred, normalize=False)

        # Calculate AIC and BIC
        AIC = 2*k - 2*log_likelihood
        BIC = -2*log_likelihood + k*math.log(len(y))

        results.append({
            'Features': ', '.join(features),
            'AIC': AIC,
            'BIC': BIC
        })

    # Convert results into a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Getting all combinations of features with at least 1 feature
features = ['x1', 'x2', 'x3']
feature_combinations = [combo for r in range(1, len(features)+1) for combo in combinations(features, r)]

# Calculate AIC and BIC for each model
aic_bic_df = calculate_aic_bic(data, 'y', feature_combinations)

# Print the results
print(aic_bic_df.sort_values(by=['AIC', 'BIC']))


     Features         AIC         BIC
1          x2  141.113016  146.323356
2          x3  142.377498  147.587838
0          x1  142.400940  147.611280
3      x1, x2  143.077195  150.892706
5      x2, x3  143.090467  150.905977
4      x1, x3  144.332480  152.147990
6  x1, x2, x3  145.063250  155.483931


In [30]:
# not working
import concurrent.futures


# Function to calculate AIC and BIC for a given set of features
def calculate_aic_bic(features):
    try:
        X = df[list(features)]
        y = df['Churn_Label']
        #X = df.loc[:, df.columns != 'Churn_Label']
        #y = df.loc[:, df.columns == 'Churn_Label']
        # Fit logistic regression model
        model = LogisticRegression(max_iter=1000)
        model.fit(X, y)

        # Number of parameters (features + intercept)
        k = len(features) + 1

        # Calculate log-likelihood
        y_pred = model.predict_proba(X)
        log_likelihood = -log_loss(y, y_pred, normalize=False)

        # Calculate AIC and BIC
        AIC = 2*k - 2*log_likelihood
        BIC = -2*log_likelihood + k*math.log(len(y))

        return {
            'Features': ', '.join(features),
            'AIC': AIC,
            'BIC': BIC
        }
    except Exception as e:
        print(f"Error with features {features}: {e}")
        return None

# Getting all combinations of features with at least 1 feature
#X = df.loc[:, df.columns != 'Churn_Label']
#y = df.loc[:, df.columns == 'Churn_Label']


features = df.columns.drop('Churn_Label')
feature_combinations = [combo for r in range(1, len(features)+1) for combo in combinations(features, r)]

# Calculate AIC and BIC for each model using parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(calculate_aic_bic, feature_combinations))

# Convert results into a DataFrame and sort by AIC and BIC
results_df = pd.DataFrame(results)
print(results_df.sort_values(by=['AIC', 'BIC']))

KeyboardInterrupt: 